<a href="https://colab.research.google.com/github/veroniquekop/Dataton-2022/blob/main/Code_mapping_tissue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv
import requests
import time


def get_mapping_from_file(tissue_code):
    with open('code_mapping.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if row[2].lower() == tissue_code.lower():
                return row[1]
    return None


def get_mapping_from_api(tissue_code):
    response = requests.get(f"http://oncotree.mskcc.org/api/tumorTypes/search/code/{tissue_code}?exactMatch=true&levels=2%2C3%2C4%2C5")
    json_data = response.json()
    
    if json_data and isinstance(json_data, list) and len(json_data) > 0:
        tissue = json_data[0].get("tissue")
        
        time.sleep(2)
        
        response = requests.get(f"http://oncotree.mskcc.org/api/tumorTypes/search/name/{tissue}?exactMatch=true&levels=1")
        json_data = response.json()

        if tissue in special_cases:
            mapping = get_mapping_from_file(special_cases[tissue])
            if mapping:
                return mapping
        
        if json_data and isinstance(json_data, list) and len(json_data) > 0:
            code_mapping = json_data[0].get("code")
            mapping = get_mapping_from_file(code_mapping)
            if mapping:
                return mapping
    
    return None


def get_mapping(code):
    mapping = get_mapping_from_file(code)
    
    if mapping:
        return mapping
    
    mapping = get_mapping_from_api(code)
    
    if mapping:
        return mapping
    
    return None


code = input("Введите код опухоли: ")
special_cases = {
    "Esophagus/Stomach": "STOMACH",
    "Bladder/Urinary Tract": "BLADDER",
    "Ovary/Fallopian Tube": "OVARY",
    "Vulva/Vagina": "VULVA"
   
}

mapping = get_mapping(code)

if mapping:
    print(mapping)
else:
    print("Мэппинг не найден.")


Введите код опухоли: gb
Мэппинг не найден.
